# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: I think this is a classification problem because we are classifying students into two categories: (1) the ones who need an intervention and (2) the ones who do not need intervention because they are not at risk to fail. Logistic regression can work as well here, because we logistic regression when the output is categorical. However using logistic regression does not make this problem a regression problem. This is a classsification problem.**

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [34]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from IPython.display import display

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [18]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed']=='yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed']=='no'])

# TODO: Calculate graduation rate
grad_rate = (float(n_passed)/(n_students))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [19]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [20]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [21]:
# TODO: Import any additional functionality you may need here

from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=40)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer:  I chose 1) decision trees, 2) K-nearest neighbors, and 3) Support Vector Machine


A) Decision trees example: 1) A decision tree was created to improve intermediate care for patients admitted to Cristiana Health Care System in Newark, Delaware. Patients were allowed to stay in the same bed from admission to discharge regardless of the type of care they needed. Patients who were coming to intermediate care from emergency rooms met the criteria to participate in this pilot system based on the newly developed decision tree. The waiting time was reduced by 50% for patients from emergency room, the costs for nurses were reduced, and the average waiting time for all patients was reduced by 80%. The decision tree was created by an interdisciplinary team based on 'InterQual' criteria.  Article title: Appropriate Admissions to the Appropriate Units: A decision tree approach, Authors: Joanne Matukaitis, Paula Stillman, Elizabeth Wykpisz, and Edward Ewen. Published at American Journal of Medical Quality, 2005, V. 20, Issue 2. 

Strengths of decision trees: 1) can be truly great predictors for cases where the training and the testing data do not have much variety (but are similar). 2) Decision trees are faithful to the training data and that is why they tend to overfit. 3) Decision trees are easy to interpret and can even indicate which of the inputs are the best predictors for the output. 4) We do not have to worry about linearity (such as in the case of logistic regression). 5) we can play with how deep or shallow we want the decision tree to be in order to get a better predictor. 6) they are time effective in creating classification tables directly from the data

Eeaknesses of decision trees: 1) it tends to overfit the training data which makes it more challenging to generalize among unknown data such as testing data.  2) they can be very complex which may require some degree of pruning once the decision tree is created. 3) decision trees may stuck into minutia classifications because of the tendency to truly adjust to the training data even if we have pruned it. 

Reason for choosing it: This is a relatively small dataset with many features. Decision trees work best with larger dataset and fewer features because of their tendency to overfit the training data.  However, decision trees are truly good predictors when the input variables are a mix of categorical and numerical variables (such as this case). In addition, decision trees are very useful in gaining familiarity with the data.  Finally, in this case we are using three different methods, so decision trees will be a good comparison methods. 



B) K-Nearest neighbors example: This methods was used to try to improve detecting face recognition in security systems at lower power processors. The proposed algorythm was tested on three different datasets previously used with high speed processors. The KNN algorythm worked better at a lower power processor detecting 91.5% with k=1. Article title: Implementation of K-Nearest Neighbors Face Recognition on Low-power Processor, authors: Eko Setiawan & Adharul Muttaqin. Published at TELKOMNIKA, 2015, Volume 13, Nr. 3

Strengths of k-nearest neighbors: 1) no training time is needed so it is time effective. 2) very robust against noise which makes knn a very good candidate to compare with decision trees. 3) shows adaptive behavior because looks for nearby similarities. 

Weaknesses of k-nearest neighbors: 1)it is sensitive to the points we have chosen because it looks for similarities so it may have biases 2) k-nearest can be slower if we have a large amount of training data. 

Reasons for selecting this method: I chose K-nearest neighbor for two basic reasons: 1) because our training data is small, so the KNN should perform well and 2) because it is a very different methods from decision trees, the comparison should be more interesting than comparing similar methods. 


C) Support Vector Machines were used to predict students' problem solving performance in computer based learning environments. The SVM model using rbf, polynomial, linear, and sigmoid kernels were used to estimate the performance level of students before they utilized hints available to them for problem solving. SVM was used to see whether it did a better job that bayesian knowledge tracking system that was used until that time. SVM seemed to show superior prediction level than bayesian knowledge (at 29% improvement). Article title: Predicting Students’ Problem Solving Performance using Support Vector Machine, author: Young-Jin Lee, published at Journal of Data Science, volume 14, 2016.  

Strengths: 1) very effective classification method that can handle many features (like our dataset), 2) it is flexibility in terms of kernel (we can tune them until we find the one that works best), 3) it works good with some noise but it is sensitive to a lot of noise

Weaknesses: 1) it does not work well for datasets where nr of features > than the nr of cases. 2) it can overfit the data which may create problems with generalization

Reasons for choosing this method: SVM seemed like the perfect choice for this dataset (where the nr of features < than the nr of cases). It also seems to be a good range of variability of these three methods where each methods offer different strengths and weaknesses that can be compared to each other.  **

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [22]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [29]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C


from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize the three models
clf_A = KNeighborsClassifier()
clf_B = DecisionTreeClassifier(random_state=0)
clf_C = SVC()



# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

for clf in [clf_A, clf_B, clf_C]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
        print '/n'


Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0014 seconds
Made predictions in 0.0012 seconds.
F1 score for training set: 0.8477.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.7413.
/n
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0005 seconds
Made predictions in 0.0025 seconds.
F1 score for training set: 0.8211.
Made predictions in 0.0014 seconds.
F1 score for test set: 0.7448.
/n
Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0013 seconds
Made predictions in 0.0059 seconds.
F1 score for training set: 0.8505.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7838.
/n
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0007 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6500.
/n
Training a DecisionTreeCla

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - DecisionTreeClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0011           |   0.0001                  |   1.0000               |  0.6560          |
| 200               |      0.0018          |    0.0002                 |   1.0000               |   0.7200              
| 300               |      0.0020         |     0.0002                |    1.0000             |   0.7669      |

** Classifer 2 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0007           |        0.0014            |       0.8235           |    0.8199           |
| 200               |    0.0006          |         0.0017         |          0.7962        |      0.8153            |
| 300               |       0.0006         |       0.0024             |     0.8243          |   0.8158    |

** Classifer 3 - Support Vector Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0010                  |     0.0006               |   0.8750         |    0.8199      |
| 200               |       0.0029                  |      0.0013              |  0.8664        |      0.7895     |
| 300               |        0.0046                |        0.0016             |   0.8786         |    0.8366          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: I choose Support Vector Classifier as the best option from the three, mostly because of its ability to best predict the outcome. Evaluating the time it took for training and predicting, SVM may be the most time consuming but the difference is relatively small and it weighs towards its higher accuracy. **

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: After trying three different models, it seems that the model called Support Vector Machine is the best to predict the probability of students who are likely to fail or pass.  This type of classification model has the ability to divide the data into two groups: 1) failing students and 2) passing students while considering all the characteristics of students we entered in the model. It also gave us the highest F1 score. F1 score is a measure of accuracy or the ratio of the data that was accurately predicted. The closer the F-score is to 1 the best the prediction is and the closer to 0 it is, the worse the prediction. That means that our score of .8366 is pretty decent.  

The truly nice things about Support Vector Machine is that it does not simply divide the data into two categories of our choice, but it also gives us the best line that divides the data by maximizing the distance from each group. The figure below illustrates this point. As you can see the dataset is divided into two groups (the green dots representing one group and the red dots representing the other). For illustrating this dataset, we can think of one group as the passing students and the other as the failing students. In addition, the dotted line in the middle between support vectors shows the best fitted line that has maximized the distance between the two groups.  This ability of Support Vector Machine model makes it preferable for the type of data we have. 

Figure 1: Illustration of Support Vector Machine

![Image of SVM](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/8650/1461641603/SVM_2.png)




 



**

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [52]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import svm, grid_search, datasets
from sklearn.metrics import make_scorer,f1_score
from sklearn.grid_search import GridSearchCV

# TODO: Create the parameters list you wish to tune
parameters = {"C": [pow (2, c) for c in range(-4, 3)],'kernel' : ['linear', 'rbf']}


# TODO: Initialize the classifier
clf = SVC(random_state=20)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, parameters, pos_label = 'yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0026 seconds.
Tuned model has a training F1 score of 0.8188.
Made predictions in 0.0011 seconds.
Tuned model has a testing F1 score of 0.8366.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: The F1 score here is the same as the untuned model = .8366.  However, I worked in a separate file on this same project and because of the way data was separated, I got an initial lower F1 score and a better F1 score after tuning.  This seems to be different when I started everything from the beginning because the data was randomly split differently this time. To tune it, I played around with the margins as well as kernels and tried many different combinations until I got this one that seems to produce the best results. **

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.